In [354]:
import pandas
import scipy
from scipy.stats import fisher_exact

pandas.options.display.max_columns=999

Load mutations data set

In [367]:
MUTATIONS = pandas.read_pickle('tables/MUTATIONS.pkl.gz')
MUTATIONS.reset_index(inplace=True)
MUTATIONS = MUTATIONS[(MUTATIONS.IS_FILTER_PASS) & (~MUTATIONS.IS_HET) & (~MUTATIONS.IS_NULL)]
MUTATIONS[:4]

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,IS_INDEL,IN_CDS,IN_PROMOTER,IS_SYNONYMOUS,IS_NONSYNONYMOUS,IS_HET,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES
0,site.02.subj.0958.lab.22A197.iso.1,rpoB,P45S,45.0,45.0,NaN,NaN,ccg,tcg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1
1,site.02.subj.0958.lab.22A197.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1
2,site.02.subj.0958.lab.22A197.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1
3,site.02.subj.0958.lab.22A197.iso.1,rpoC,D271E,271.0,271.0,NaN,NaN,gac,gag,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1


Load effects data set

In [275]:
EFFECTS = pandas.read_pickle('tables/EFFECTS.pkl.gz')
EFFECTS.reset_index(inplace=True)
EFFECTS[:20]

,UNIQUEID,DRUG,GENE,MUTATION,CATALOGUE_NAME,CATALOGUE_VERSION,CATALOGUE_GRAMMAR,SITEID,PREDICTION,DEFAULT_CATALOGUE
0,site.02.subj.0958.lab.22A197.iso.1,RFB,rpoB,P45S,CRyPTIC,v1.31,GARC1,02,U,True
1,site.02.subj.0958.lab.22A197.iso.1,RIF,rpoB,P45S,CRyPTIC,v1.31,GARC1,02,U,True
2,site.02.subj.0958.lab.22A197.iso.1,RFB,rpoB,S450L,CRyPTIC,v1.31,GARC1,02,U,True
3,site.02.subj.0958.lab.22A197.iso.1,RIF,rpoB,S450L,CRyPTIC,v1.31,GARC1,02,R,True
4,site.02.subj.0958.lab.22A197.iso.1,RFB,rpoB,A1075A,CRyPTIC,v1.31,GARC1,02,S,True
5,site.02.subj.0958.lab.22A197.iso.1,RIF,rpoB,A1075A,CRyPTIC,v1.31,GARC1,02,S,True
6,site.02.subj.0958.lab.22A197.iso.1,RFB,rpoC,D271E,CRyPTIC,v1.31,GARC1,02,U,True
7,site.02.subj.0958.lab.22A197.iso.1,RIF,rpoC,D271E,CRyPTIC,v1.31,GARC1,02,U,True
8,site.02.subj.0823.lab.2013241494.iso.1,RIF,rpoB,A1075O,CRyPTIC,v1.31,GARC1,02,U,True
9,site.02.subj.0823.lab.2013241494.iso.1,RFB,rpoB,A1075O,CRyPTIC,v1.31,GARC1,02,U,True


Find all samples with resistance mutations in EFFECTS and save UNIQUEID

In [17]:
res_samples = EFFECTS[(EFFECTS.PREDICTION=='R') & (~EFFECTS.MUTATION.str[-1].isin(['O','X']))].UNIQUEID
res_samples.head()

3         site.02.subj.0958.lab.22A197.iso.1
16        site.02.subj.0918.lab.22A153.iso.1
45    site.02.subj.1033.lab.2013185075.iso.1
51    site.02.subj.0739.lab.2013221518.iso.1
89        site.02.subj.0104.lab.22A057.iso.1
Name: UNIQUEID, dtype: object

Get all mutations and corresponding genes from samples that have one or more resistance mutations

In [397]:
co_occurr=pandas.DataFrame()
co_occurr['mutation'] = MUTATIONS.loc[MUTATIONS['UNIQUEID'].isin(res_samples)].MUTATION
co_occurr['gene']=MUTATIONS.loc[MUTATIONS['UNIQUEID'].isin(res_samples)].GENE
co_occurr.value_counts()

mutation  gene
S450L     rpoB    13609
A1075A    rpoB    12240
A542A     rpoC     2880
E1092D    rpoC     2439
G594E     rpoC     1864
                  ...  
R665W     rpoB        1
E18E      rpoC        1
K837T     rpoC        1
K878K     rpoB        1
P917P     rpoB        1
Length: 3177, dtype: int64

Remove mutations from list that are resistance mutations

In [53]:
res_mut = pandas.DataFrame()
res_mut['mutation'] = EFFECTS[(EFFECTS.PREDICTION=='R') & (~EFFECTS.MUTATION.str[-1].isin(['O','X']))].MUTATION
res_mut['gene'] = EFFECTS[(EFFECTS.PREDICTION=='R') & (~EFFECTS.MUTATION.str[-1].isin(['O','X']))].GENE
#here: think about mutations gene locations TODO, maybe this messes up ratios?
co_occurr_only = co_occurr.loc[~co_occurr['mutation'].isin(res_mut['mutation'])]
co_occurr_only.value_counts()[:20]

mutation  gene
A1075A    rpoB    12240
A542A     rpoC     2880
E1092D    rpoC     2439
G594E     rpoC     1864
G876G     rpoB     1458
c-61t     rpoB     1402
V483G     rpoC     1236
D103D     rpoB     1041
I491V     rpoC      666
V483A     rpoC      599
R173R     rpoC      595
A172V     rpoC      589
I491T     rpoC      457
P1040R    rpoC      400
F452S     rpoC      346
C62C      rpoC      337
E761D     rpoB      304
P601L     rpoC      248
E561E     rpoB      229
L731P     rpoB      226
dtype: int64

**Either** look only at co-occuring mutations that do not occur without resistance mutations:

- first determine how often mutations that co-occurr are found in general

In [151]:
gen_occurr=pandas.DataFrame()
gen_occurr['mutation']=MUTATIONS.loc[MUTATIONS['MUTATION'].isin(co_occurr_only.mutation)].MUTATION
gen_occurr['gene']=MUTATIONS.loc[MUTATIONS['MUTATION'].isin(co_occurr_only.mutation)].GENE
gen_occurr.value_counts()

mutation  gene
A1075A    rpoB    37400
G594E     rpoC    10610
A542A     rpoC    10544
G876G     rpoB     9236
c-61t     rpoB     8752
                  ...  
S401P     rpoC        1
F438L     sigA        1
F424I     rpoB        1
F424C     rpoB        1
E830Q     rpoC        1
Length: 3149, dtype: int64

In [263]:
df_gen=gen_occurr.value_counts().reset_index(name='count')
df_res=co_occurr_only.value_counts().reset_index(name='count')
df_gen
#gen_occurr.value_counts()-co_occurr_only.value_counts()

,mutation,gene,count
0,A1075A,rpoB,37400
1,G594E,rpoC,10610
2,A542A,rpoC,10544
3,G876G,rpoB,9236
4,c-61t,rpoB,8752
...,...,...,...
3144,S401P,rpoC,1
3145,F438L,sigA,1
3146,F424I,rpoB,1
3147,F424C,rpoB,1


- then identify if difference of the general occurrence and occurrence with resistance mutations is zero, if yes, mutations only occurr with resistance mutations

In [440]:
#df_gen.set_index('mutation')
#df_res.set_index('mutation')
df_gen.sort_values('mutation',inplace=True)
df_res.sort_values('mutation',inplace=True)

df_gen['diff']=df_gen['count']-df_res['count']
df_gen[(df_gen['diff']<1)]
#df_gen[(df_gen['diff']==0)&(df_gen['count']>1)]

,mutation,gene,count,diff
2619,-11_indel,rpoZ,1,0.0
2617,-3_indel,rpoA,1,0.0
2632,1023_indel,rpoA,1,0.0
2624,1286_indel,rpoB,1,0.0
2744,1361_indel,rpoB,1,0.0
...,...,...,...,...
2637,g-60a,rpoZ,1,0.0
2640,g-7a,sigA,1,0.0
2611,g-92t,rpoB,1,0.0
2626,t-45g,rpoB,1,0.0


**Or** determine if co-occurrence with resistance is significant compared to general occurrence in mutations dataset:

- find number of samples in mutations set

In [273]:
len(MUTATIONS.UNIQUEID.value_counts())

64722

- find number of samples with resistance mutations in effects dataset

In [324]:
len(EFFECTS[(EFFECTS.PREDICTION=='R')].UNIQUEID.value_counts())

21046

- add columns with resistance indication and co-occurence mutation indication to mutations data frame  (I decided to classify U as S for now)

In [368]:
#only use the first line once
#MUTATIONS['resistance']=False
MUTATIONS.loc[MUTATIONS['UNIQUEID'].isin(EFFECTS[(EFFECTS.PREDICTION=='R')].UNIQUEID),'resistance']=True
MUTATIONS[:20]

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,IS_INDEL,IN_CDS,IN_PROMOTER,IS_SYNONYMOUS,IS_NONSYNONYMOUS,IS_HET,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,resistance
0,site.02.subj.0958.lab.22A197.iso.1,rpoB,P45S,45.0,45.0,NaN,NaN,ccg,tcg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True
1,site.02.subj.0958.lab.22A197.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True
2,site.02.subj.0958.lab.22A197.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,True
3,site.02.subj.0958.lab.22A197.iso.1,rpoC,D271E,271.0,271.0,NaN,NaN,gac,gag,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True
6,site.02.subj.0359.lab.222018-14.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,False
7,site.02.subj.0224.lab.2013221088.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,False
8,site.02.subj.0918.lab.22A153.iso.1,rpoB,D435G,435.0,435.0,NaN,NaN,gac,ggc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True
9,site.02.subj.0918.lab.22A153.iso.1,rpoB,I491L,491.0,491.0,NaN,NaN,atc,ctc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True
10,site.02.subj.0918.lab.22A153.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,True
16,site.02.subj.0045.lab.2015231012.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,False


In [369]:
#only use the first line once
#here: think about mutations gene locations
#MUTATIONS['co_occurance']=False
MUTATIONS.loc[MUTATIONS['MUTATION'].isin(co_occurr_only.mutation),'co_occurance']=True
MUTATIONS[:20]

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,IS_INDEL,IN_CDS,IN_PROMOTER,IS_SYNONYMOUS,IS_NONSYNONYMOUS,IS_HET,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES,resistance,co_occurance
0,site.02.subj.0958.lab.22A197.iso.1,rpoB,P45S,45.0,45.0,NaN,NaN,ccg,tcg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True,True
1,site.02.subj.0958.lab.22A197.iso.1,rpoB,S450L,450.0,450.0,NaN,NaN,tcg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True,False
2,site.02.subj.0958.lab.22A197.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,True,True
3,site.02.subj.0958.lab.22A197.iso.1,rpoC,D271E,271.0,271.0,NaN,NaN,gac,gag,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True,True
6,site.02.subj.0359.lab.222018-14.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,False,True
7,site.02.subj.0224.lab.2013221088.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,False,True
8,site.02.subj.0918.lab.22A153.iso.1,rpoB,D435G,435.0,435.0,NaN,NaN,gac,ggc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True,False
9,site.02.subj.0918.lab.22A153.iso.1,rpoB,I491L,491.0,491.0,NaN,NaN,atc,ctc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,,,02,1,True,True
10,site.02.subj.0918.lab.22A153.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,True,True
16,site.02.subj.0045.lab.2015231012.iso.1,rpoB,A1075A,1075.0,1075.0,NaN,NaN,gct,gcc,True,False,True,False,True,False,False,False,True,GENE,AAM,NaN,,,02,1,False,True


- concatenate rows per sample (True/False fro resistance and co-occurance of mutations)

In [401]:
summary_statistics=MUTATIONS.groupby('UNIQUEID').sum()
summary_statistics

,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,IS_SNP,IS_INDEL,IN_CDS,IN_PROMOTER,IS_SYNONYMOUS,IS_NONSYNONYMOUS,IS_HET,IS_NULL,IS_FILTER_PASS,INDEL_LENGTH,NUMBER_NUCLEOTIDE_CHANGES,resistance,co_occurance
UNIQUEID,,,,,,,,,,,,,,,,,
site.00.subj.1000347.lab.H111540004.iso.1,3378.0,3378.0,0.0,0.0,4,0,4,0,1,3,0,0,4,0.0,4,4,3
site.00.subj.1000595.lab.H123460044.iso.1,1977.0,1977.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,3,2
site.00.subj.1004213.lab.H111060034.iso.1,2777.0,2777.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,3,2
site.00.subj.1004213.lab.H112000008.iso.1,2777.0,2777.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,3,2
site.00.subj.1004213.lab.H113100007.iso.1,2777.0,2777.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
site.QC.subj.N1176.lab.1.iso.1,2238.0,2238.0,0.0,0.0,3,0,3,0,3,0,0,0,3,0.0,3,0,3
site.QC.subj.N1177.lab.1.iso.1,2808.0,2808.0,0.0,0.0,5,0,5,0,1,4,0,0,5,0.0,5,5,4
site.QC.subj.N1202.lab.1.iso.1,2102.0,2102.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,0,3


In [445]:
summary_statistics.loc[summary_statistics['resistance']!=0,'resistance']=True
summary_statistics.loc[summary_statistics['co_occurance']!=0,'co_occurance']=True
summary_statistics

,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,IS_SNP,IS_INDEL,IN_CDS,IN_PROMOTER,IS_SYNONYMOUS,IS_NONSYNONYMOUS,IS_HET,IS_NULL,IS_FILTER_PASS,INDEL_LENGTH,NUMBER_NUCLEOTIDE_CHANGES,resistance,co_occurance
UNIQUEID,,,,,,,,,,,,,,,,,
site.00.subj.1000347.lab.H111540004.iso.1,3378.0,3378.0,0.0,0.0,4,0,4,0,1,3,0,0,4,0.0,4,True,True
site.00.subj.1000595.lab.H123460044.iso.1,1977.0,1977.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,True,True
site.00.subj.1004213.lab.H111060034.iso.1,2777.0,2777.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,True,True
site.00.subj.1004213.lab.H112000008.iso.1,2777.0,2777.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,True,True
site.00.subj.1004213.lab.H113100007.iso.1,2777.0,2777.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
site.QC.subj.N1176.lab.1.iso.1,2238.0,2238.0,0.0,0.0,3,0,3,0,3,0,0,0,3,0.0,3,0,True
site.QC.subj.N1177.lab.1.iso.1,2808.0,2808.0,0.0,0.0,5,0,5,0,1,4,0,0,5,0.0,5,True,True
site.QC.subj.N1202.lab.1.iso.1,2102.0,2102.0,0.0,0.0,3,0,3,0,1,2,0,0,3,0.0,3,0,True


- set up contigency table to be applied to Fishers exact test

In [441]:
fisher_test_set=pandas.crosstab(summary_statistics.resistance, summary_statistics.co_occurance)
fisher_test_set

co_occurance,0,True
resistance,,
0,810,43056
True,1008,19848


In [405]:
oddsr, p = fisher_exact(fisher_test_set, alternative='two-sided')
print(oddsr, p)

0.37043119923554707 5.998685659432233e-95


**Or** determine relative occurrence of mutations in co-occurr list in general mutations set and in resistant samples

In [451]:
rel_occurr_res=co_occurr_only.value_counts().div(21046).reset_index(name='resistant_ratio')
rel_occurr_gen=gen_occurr.value_counts().div(64722).reset_index(name='general_ratio')
print(rel_occurr_res[:20])
print(rel_occurr_gen[:20])

   mutation  gene  resistant_ratio
0    A1075A  rpoB         0.581583
1     A542A  rpoC         0.136843
2    E1092D  rpoC         0.115889
3     G594E  rpoC         0.088568
4     G876G  rpoB         0.069277
5     c-61t  rpoB         0.066616
6     V483G  rpoC         0.058728
7     D103D  rpoB         0.049463
8     I491V  rpoC         0.031645
9     V483A  rpoC         0.028461
10    R173R  rpoC         0.028271
11    A172V  rpoC         0.027986
12    I491T  rpoC         0.021714
13   P1040R  rpoC         0.019006
14    F452S  rpoC         0.016440
15     C62C  rpoC         0.016013
16    E761D  rpoB         0.014445
17    P601L  rpoC         0.011784
18    E561E  rpoB         0.010881
19    L731P  rpoB         0.010738
   mutation  gene  general_ratio
0    A1075A  rpoB       0.577856
1     G594E  rpoC       0.163932
2     A542A  rpoC       0.162912
3     G876G  rpoB       0.142703
4     c-61t  rpoB       0.135224
5     D103D  rpoB       0.104400
6     R173R  rpoC       0.103643
7

In [459]:
concat_rel_occurr=pandas.DataFrame()
rel_occurr_res.sort_values('mutation',inplace=True)
rel_occurr_gen.sort_values('mutation',inplace=True)

concat_rel_occurr=rel_occurr_res
concat_rel_occurr['general_ratio']=rel_occurr_gen['general_ratio']
concat_rel_occurr[(concat_rel_occurr['resistant_ratio']-concat_rel_occurr['general_ratio']>0.002)]

,mutation,gene,resistant_ratio,general_ratio
0,A1075A,rpoB,0.581583,0.577856
11,A172V,rpoC,0.027986,0.019205
15,C62C,rpoC,0.016013,0.010568
26,D485N,rpoC,0.007887,0.004991
21,D485Y,rpoC,0.009455,0.007215
33,E319K,rpoA,0.005512,0.003198
16,E761D,rpoB,0.014445,0.010136
14,F452S,rpoC,0.016440,0.014400
32,G332R,rpoC,0.005559,0.003229
28,G433S,rpoC,0.006700,0.003507


Compare known compensatory mutations to list of hits

In [457]:
print(concat_rel_occurr[(concat_rel_occurr['mutation']=='L516P')])
print(concat_rel_occurr[(concat_rel_occurr['mutation']=='R186C')])
print(concat_rel_occurr[(concat_rel_occurr['mutation']=='D485N')])
print(concat_rel_occurr[(concat_rel_occurr['mutation']=='T187A')])
print(concat_rel_occurr[(concat_rel_occurr['mutation']=='T187P')])

   mutation  gene  resistant_ratio  general_ratio
27    L516P  rpoC         0.006937       0.004697
     mutation  gene  resistant_ratio  general_ratio
2558    R186C  rpoA         0.000048       0.000031
   mutation  gene  resistant_ratio  general_ratio
26    D485N  rpoC         0.007887       0.004991
   mutation  gene  resistant_ratio  general_ratio
25    T187A  rpoA         0.008173       0.005361
    mutation  gene  resistant_ratio  general_ratio
212    T187P  rpoA         0.000713       0.001993
